# Exercise - SVM

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since you have a relatively small data set, I recommend using cross-validation to evaluate your accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(12)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

##  Identify the numeric, binary, and categorical columns

In [6]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [7]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [8]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [9]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [10]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [11]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [12]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[ 0.30124253,  0.02444976, -0.03976584, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.66145438, -0.48267317, -0.20148348, ...,  0.        ,
         0.        ,  0.        ],
       [-0.41039551, -0.30168367, -0.47126737, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.3856032 , -1.03853627, -1.0206215 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.88109575,  1.5990853 ,  1.46844204, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.37056539, -1.19752907,  0.07241276, ...,  0.        ,
         0.        ,  1.        ]])

In [13]:
train_x.shape

(1750, 20)

# Tranform: transform() for TEST

In [14]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[-0.55975164, -0.33390797, -0.27130663, ...,  0.        ,
         0.        ,  0.        ],
       [-0.62125123, -0.40462038, -0.49507387, ...,  0.        ,
         1.        ,  0.        ],
       [-0.97267742, -1.116638  , -0.85248374, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.33638515,  1.88406362,  1.63271038, ...,  0.        ,
         0.        ,  0.        ],
       [-1.13960486, -1.48610419, -1.18891543, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05896932, -0.40312783, -0.40723968, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
test_x.shape

(750, 20)

# Calculate the Baseline

In [16]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
# This is the baseline Train Accuracy

dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6017142857142858


In [19]:
# This is the baseline Test Accuracy

dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5813333333333334


# Train an SVM model with linear kernel

In [20]:
from sklearn.svm import SVC
 
lin_svm = SVC(kernel="linear")


In [21]:
lin_svm.fit(train_x, train_target)

SVC(kernel='linear')

### Calculate the accuracy

In [22]:
from sklearn.metrics import accuracy_score

In [24]:
#Predict the train values
train_target_pred = lin_svm.predict(train_x)


In [25]:
#Train accuracy
accuracy_score(train_target, train_target_pred)

0.7382857142857143

In [26]:
#Predict the test values
test_target_pred = lin_svm.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_target_pred)

0.724

# Train an SVM model with poly kernel

In [44]:
from sklearn.svm import SVC

# You need to enter a value for gamma. Remember, gamma controls the shape of the bell curve for rbf
# You can also set it is as gamma='scale'. This will be the default option in future releases

pol_svm = SVC(kernel="poly", degree=3, coef0=1, C=0.1)

pol_svm.fit(train_x, train_target)

SVC(C=0.1, coef0=1, kernel='poly')

In [45]:
#Predict the train values
train_target_pred = pol_svm.predict(train_x)


In [46]:
#Predict the test values
test_target_pred = pol_svm.predict(test_x)

### Calculate the accuracy

In [47]:
#Train accuracy
accuracy_score(train_target, train_target_pred)

0.7902857142857143

In [48]:
#Test accuracy
accuracy_score(test_target, test_target_pred)

0.7413333333333333

# Train an SVM model with rbf kernel

In [81]:
rbf_svm = SVC(kernel="rbf", C=1, gamma='scale')


In [82]:
rbf_svm.fit(train_x, train_target)

SVC(C=1)

### Calculate the accuracy

In [83]:
#Predict the train values
train_target_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_target_pred)

0.8308571428571428

In [84]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_target_pred)

0.7413333333333333

# Optional: try grid search on one of your SVM models

In [85]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 4 (2×2) combinations of hyperparameters
    {'C': [5, 15], 
     'gamma': [0.1, 0.2]}
  ]

rbf_svm = SVC(kernel="rbf", decision_function_shape='ovr')

# train across 5 folds, that's a total of 4*5=20 rounds of training 
grid_search = GridSearchCV(rbf_svm, param_grid, cv=5,
                           scoring='accuracy', return_train_score=True)

grid_search.fit(train_x, train_target)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [5, 15], 'gamma': [0.1, 0.2]}],
             return_train_score=True, scoring='accuracy')

In [87]:
grid_search.best_params_

{'C': 15, 'gamma': 0.2}

In [88]:
grid_search.best_estimator_

SVC(C=15, gamma=0.2)

In [90]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8257142857142858 {'C': 5, 'gamma': 0.1}
0.8480000000000001 {'C': 5, 'gamma': 0.2}
0.836 {'C': 15, 'gamma': 0.1}
0.8565714285714285 {'C': 15, 'gamma': 0.2}


In [91]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.123064,0.016451,0.052714,0.008606,5,0.1,"{'C': 5, 'gamma': 0.1}",0.837143,0.828571,0.837143,...,0.825714,0.011849,4,0.906429,0.895000,0.909286,0.912857,0.904286,0.905571,0.006017
1,0.142645,0.026197,0.049451,0.013450,5,0.2,"{'C': 5, 'gamma': 0.2}",0.862857,0.857143,0.862857,...,0.848000,0.016102,2,0.957857,0.959286,0.964286,0.962857,0.966429,0.962143,0.003162
2,0.130633,0.006933,0.033558,0.007620,15,0.1,"{'C': 15, 'gamma': 0.1}",0.845714,0.845714,0.860000,...,0.836000,0.019260,3,0.939286,0.935000,0.941429,0.945000,0.945714,0.941286,0.003923
3,0.131927,0.008093,0.047038,0.000223,15,0.2,"{'C': 15, 'gamma': 0.2}",0.877143,0.854286,0.854286,...,0.856571,0.010598,1,0.987857,0.987143,0.988571,0.986429,0.992143,0.988429,0.001990


In [92]:
final_model = grid_search.best_estimator_

test_predictions = final_model.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_predictions)

0.8613333333333333